In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [2]:
mnist.train.images.shape

(55000, 784)

In [4]:
np.sqrt(784)

28.0

In [5]:
# build CNN
def build_CNN_clf(x):
    x_image = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    ## conv1 layer ##
    # 필터 shape 설정(초기랜덤값 설정할 때 표준편차 설정)
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 32],
                                             stddev=5e-2))
    # bias 설정
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
    # h_conv1
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1,
                                     strides=[1, 1, 1, 1],
                                     padding='SAME') + b_conv1)
    # h_conv1을 max pooling
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    
    ## conv2 layer ##
    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 32, 64],
                                             stddev=5e-2))
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2,
                                     strides=[1, 1, 1, 1],
                                     padding='SAME') + b_conv2)
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    
    ## Fully connected layer1 ##
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[7*7*64, 1024],
                                           stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
    # h_pool2를 flatten 시켜주기(풀링한 후 이미지 가로*세로*필터개수)
    h_pool2_flat = tf.reshape(h_pool2, shape=[-1, 7*7*64])
    # flatten시킨거랑 W_fc1 내적해서 활성함수 적용
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    ## Fully connected layer2 ## - 최종 예측값 출력시키는 layer
    W_fc2 = tf.Variable(tf.truncated_normal(shape=[1024, 10],
                                           stddev=5e-2))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
    # h_fc1과 W_fc2 내적해서 softmax(활성함수) 사용해 최종 예측값 출력
    # logits과 예측값 개별 변수에 할당
    logits = tf.matmul(h_fc1, W_fc2) + b_fc2
    y_pred = tf.nn.softmax(logits)
    
    return logits, y_pred    

In [17]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

logits, y_pred = build_CNN_clf(x)

#loss function 
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,
                                                 logits=logits)
# SGD 
sgd = tf.train.AdamOptimizer(learning_rate=0.002).minimize(loss)

# 예측값 vs 실제값 정답 여부 확인
correct_pred = tf.cast(tf.equal(tf.arg_max(y_pred, 1), tf.arg_max(y, 1)),
                      dtype=tf.float32)
# 정확도 측정
accuracy = tf.reduce_mean(correct_pred)

# 만든 Tensor들 Run
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1001):
        # 데이터 50개씩 batch size 설정
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            # 100번 iteration 돌 때마다 loss값, accuracy값 출력
            train_loss, train_acc = sess.run([loss, accuracy],
                                            feed_dict={x:batch[0],
                                                      y:batch[1]})
            train_loss = np.mean(train_loss)
            print(f"Epoch {i}번째 Loss:{train_loss:.4f} Accuracy:{train_acc:.4f}")
        # SGD으로 loss값 최소화 시키면서 파라미터 업데이트
        sess.run([sgd], feed_dict={x:batch[0], y:batch[1]})
    
    # 모든 학습이 끝나고 Test 데이터로 Loss, Accuracy 측정
    test_loss, test_acc = sess.run([loss, accuracy],
                                  feed_dict={x:mnist.test.images,
                                            y:mnist.test.labels})
    test_loss = np.mean(test_loss)
    print(f"Test 데이터 검증 결과- Loss:{test_loss:.4f} Accuracy:{test_acc:.4f}")            

Epoch 0번째 Loss:2.3827 Accuracy:0.1200
Epoch 100번째 Loss:0.2711 Accuracy:0.9200
Epoch 200번째 Loss:0.3266 Accuracy:0.8800
Epoch 300번째 Loss:0.1676 Accuracy:0.9600
Epoch 400번째 Loss:0.0634 Accuracy:0.9800
Epoch 500번째 Loss:0.1043 Accuracy:0.9800
Epoch 600번째 Loss:0.1886 Accuracy:0.9600
Epoch 700번째 Loss:0.0819 Accuracy:0.9600
Epoch 800번째 Loss:0.0362 Accuracy:1.0000
Epoch 900번째 Loss:0.0254 Accuracy:1.0000
Epoch 1000번째 Loss:0.0668 Accuracy:0.9800
Test 데이터 검증 결과- Loss:0.1020 Accuracy:0.9665


---

In [14]:
import tensorflow as tf
from tensorflow.keras.datasets.cifar10 import load_data
(x_train, y_train), (x_test, y_test) = load_data()

In [15]:
# [batch_size, height, width, 컬러
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [26]:
50000 // 200

250

In [42]:
# Build CNN
def build_CNN_clf(x):
    # x가 애초에 4차원 shape=[데이터개수, 높이, 너비, 채널개수(컬러)]로 됨
    x_image = x
    
    ### conv1 layer ### - 컬러니까 3!
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64],
                                             stddev=5e-2))
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1,
                                     strides=[1, 1, 1, 1],
                                     padding='SAME') + b_conv1)
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    #### conv2 layer ###
    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 128],
                                             stddev=5e-2))
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2,
                                     strides=[1, 1, 1, 1],
                                     padding='SAME') + b_conv2)
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    
    ### conv3 layer ###
    W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 256],
                                             stddev=5e-2))
    b_conv3 = tf.Variable(tf.constant(0.1, shape=[256]))
    h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3,
                                     strides=[1, 2, 2, 1],
                                     padding='SAME'))
    ### conv4 layer ###
    W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 256, 256]))
    b_conv4 = tf.Variable(tf.constant(0.1, shape=[256]))
    h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4,
                                     strides=[1, 2, 2, 1],
                                     padding='SAME'))
    
    ### FC layer 1 ###
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[8*8*256, 1024],
                                           stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1 , shape=[1024]))
    # h_conv4 flatten 시키기
    h_conv4_flat = tf.reshape(h_conv4, shape=[-1, 8*8*256])
    # flatten시킨 거랑 W_fc1 내적하고 활성함수 적용
    h_fc1 = tf.nn.relu(tf.matmul(h_conv4_flat, W_fc1) + b_fc1)
    
    # Dropout 적용
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    ### FC layer 2 ### - 최종 예측값 출력하는 layer
    W_fc2 = tf.Variable(tf.truncated_normal(shape=[1024, 10],
                                           stddev=5e-2))
    b_fc2 = tf.Variable(tf.constant(0.1 , shape=[10]))
    # 예측값 계산하는 logits
    logits = tf.matmul(h_fc1, W_fc2) + b_fc2
    # softmax로 예측값 분류
    y_pred = tf.nn.softmax(logits)
    
    return logits, y_pred

In [43]:
def next_batch(num, data, labels):
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    # 두개를 반환하는데, 함수 사용시 변수를 하나로 할당하면 튜플로 담김!
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32 ,3])
y = tf.placeholder(tf.float32, shape=[None, 10])
# unit(node) 얼마나 Dropout시킬지 placeholder 정의
keep_prob = tf.placeholder(tf.float32)

(x_train, y_train), (x_test, y_test) = load_data()
# Y값 원-핫 인코딩 시켜놓기
y_train_ohe = tf.squeeze(tf.one_hot(y_train, depth=10), axis=1)
y_test_ohe = tf.squeeze(tf.one_hot(y_test, depth=10), axis=1)

logits, y_pred = build_CNN_clf(x)

#cost function 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,
                                                 logits=logits))
# SGD
sgd = tf.train.RMSPropOptimizer(learning_rate=1e-3).minimize(loss)

# 예측값 vs 실제값 일치 체크
correct_pred = tf.equal(tf.arg_max(y_pred, 1), tf.arg_max(y, 1))
# 정확도 계산
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))

# 만든 텐서 Run
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1001):
        # batch_size 설정위해 직접 정의한 함수 사용
        # 텐서사용하지 않고 직접 정의한 함수이기 때문에 텐서로 만든 y_train_ohe는 eval사용해 객체로 할당
        batch = next_batch(128, x_train, y_train_ohe.eval())
        if i % 100 == 0:
            train_loss, train_acc = sess.run([loss, accuracy],
                                            feed_dict={x:batch[0],
                                                      y:batch[1],
                                                      keep_prob:1.0})
            print(f"Epoch {i}번째 Loss:{train_loss:.4f} Accuracy:{train_acc:.4f}")
            
        # SGD수행
        sess.run([sgd], feed_dict={x:batch[0], y:batch[1],
                                  keep_prob:0.8})
    # 학습 완료하고 테스트 데이터도 batch_size로 테스트
    test_acc = 0.0
    # 테스트 데이터가 만개이니까 1000개씩 10번 테스트해서 평균 정확도 계산
    for i in range(10):
        test_batch = next_batch(100, x_test, y_test_ohe.eval())
        acc = accuracy.eval(feed_dict={x: test_batch[0],
                                      y: test_batch[1],
                                      keep_prob:1.0})
        test_acc += acc
    avg_test_acc = test_acc / 10
    print()
    print(f"Test 데이터 평균 정확도: {avg_test_acc:.4f}")
        
        

In [21]:
def test(a, b):
    return np.asarray(a), np.asarray(b)

In [22]:
prac = test([1,3,5], [2,4,6])

In [24]:
type(prac)

tuple

In [29]:
x_test.shape

(10000, 32, 32, 3)

In [31]:
10000 / 1000

10.0